
# Export Promptflow Outputs

This script is used for downloading all the run info needed and merge them together in local environments.


## Prerequisite
Make sure all required libraries are installed.Use command below to install azureml.core:

    pip install azureml-core

In [1]:
!pip install azureml-core

In [2]:
import csv
import json
import logging
import requests
from pathlib import Path
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

C:\Users\mogal\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Get workspace
Get gonnections to workspace with personal token.

In [3]:
# Get a logger which allows us to log events that occur when running the program.
logger = logging.getLogger("myLogger")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s %(message)s")
ch = logging.StreamHandler()
ch.setFormatter(formatter)
ch.setLevel(logging.DEBUG)
if not logger.handlers:
    logger.addHandler(ch)

# Get the workspace associated with your personal token and get the relavant datastore.
tenant_id = "f64c46b1-d265-46fb-b644-4a4098019d69"
subscription_id = "2f47aff2-af59-42b1-b6d7-524d6f61230c"
resource_group = "first_resource"
workspace_name = "kaggle_essayscoring"

interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)
ws = Workspace.get(workspace_name, subscription_id=subscription_id, resource_group=resource_group, auth=interactive_auth)
region = ws.location
default_datastore = ws.get_default_datastore()

## Define utils
Define functions for downloading and loading run info.

In [4]:
# Get run infor for the given run_id
def get_run_info(run_id):
    logger.info(f"Getting Run Info for Run: {run_id}")
    run = ws.get_run(run_id=run_id)
    display_name = run.display_name
    input_run_id = run.properties.get('azureml.promptflow.input_run_id')
    return display_name, input_run_id

# Get the output asset id for the given run_id and asset_name
def get_output_asset_id(run_id, asset_name):
    logger.info(f"Getting Output Asset Id for Run {run_id}")
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
    else:
        url = f"https://ml.azure.com/api/{region}/history/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/rundata"
    payload = {
        "runId": run_id,
        "selectRunMetadata": True
    }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get output asset id for run {run_id} because RunHistory API returned status code {response.status_code}. Response: {response.text}")
    output_asset_id = response.json()["runMetadata"]["outputs"][asset_name]["assetId"]
    return output_asset_id

# Get the asset path for the given asset_id
def get_asset_path(asset_id):
    logger.info(f"Getting Asset Path for Asset Id {asset_id}")
    if region == "centraluseuap":
        url = f"https://int.api.azureml-test.ms/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    else:
        url = f"https://ml.azure.com/api/{region}/data/v1.0/subscriptions/{subscription_id}/resourceGroups/{resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{workspace_name}/dataversion/getByAssetId"
    payload = {
        "value": asset_id,
    }
    response = requests.post(url, json=payload, headers=ws._auth.get_authentication_header())
    if response.status_code != 200:
        raise Exception(f"Failed to get asset path for asset id {asset_id} because Data API returned status code {response.status_code}. Response: {response.text}")
    data_uri = response.json()["dataVersion"]["dataUri"]
    relative_path = data_uri.split("/paths/")[-1]
    return relative_path

# Get the flow artifact relative path for the given run_id
def get_flow_artifact_relative_path(run_id):
    logger.info(f"Getting Flow Artifact Relative Path for Run {run_id}")
    try:
        flow_artifact_asset_id = get_output_asset_id(run_id, "debug_info")
        relative_path = get_asset_path(flow_artifact_asset_id)
        relative_path += "flow_artifacts/"
        return relative_path
    except Exception as e:
        logger.warning("`debug_info` output assets is not available, maybe because the job ran on old version runtime, trying to get `flow_outputs` output asset instead.")
        output_asset_id = get_output_asset_id(run_id, "flow_outputs")
        relative_path = get_asset_path(output_asset_id)
        return relative_path.replace("flow_outputs", "flow_artifacts")

# Download flow artifacts for the given run_id
def download_flow_artifacts(run_id, blob_prefix):
    logger.info(f"Downloading Flow Artifacts for Run {run_id}")
    target_dir = f"./downloads/{run_id}"
    default_datastore.download(target_dir, prefix=blob_prefix, overwrite=True)
    return target_dir

# Load flow artifacts of the run from the given local_dir
def load_flow_artifacts(local_dir, run_display_name):
    logger.info(f"Loading Flow Artifacts of {run_display_name} from {local_dir}")
    flow_artifacts = {}
    p = Path(local_dir)
    for file_path in p.glob("**/*.jsonl"):
        with open(file_path) as fp:
            for line in fp:
                line_record = json.loads(line)
                line_number = line_record.get("line_number")
                run_info = line_record.get("run_info") or {}
                status = run_info.get("status")
                outputs = run_info.get("output") or {}
                inputs = run_info.get("inputs") or {}
                modified_inputs = {f"inputs.{k}": v for k, v in inputs.items()}
                record = {
                    "Line number": line_number,
                    "Run": run_display_name,
                    "Status": status,
                    **modified_inputs,
                    **outputs
                }
                flow_artifacts[line_number] = record
    return flow_artifacts

# Get required info for the given run_id
def get_required_info(run_id):
    logger.info(f"Processing Run: {run_id}")
    display_name, input_run_id = get_run_info(run_id)
    flow_artifact_relative_path = get_flow_artifact_relative_path(run_id)

    target_dir = download_flow_artifacts(run_id, flow_artifact_relative_path)
    flow_artifacts = load_flow_artifacts(target_dir, display_name)
    return {
        "run_id": run_id,
        "display_name": display_name,
        "input_run_id": input_run_id,
        "flow_artifacts": flow_artifacts
    }

# Update downstream flow artifacts with the given display_name
def update_downstread_flow_artifacts(flow_artifacts, display_name):
    updated_flow_artifacts = {}
    for line_number, line_record in flow_artifacts.items():
        updated_line_record = {}
        for k, v in line_record.items():
            if k == "Run" or k == "Status" or k == "Line number":
                continue
            else:
                updated_line_record[f"{k}({display_name})"] = v
        updated_flow_artifacts[line_number] = updated_line_record
    return updated_flow_artifacts

# Merge flow artifacts for the given run_infos, joining flow artifacts with the same line number
def merge_flow_artifacts(run_infos: list):
    run_info_dict = {run_info["run_id"]: run_info for run_info in run_infos}
    main_flow_infos = []
    for run_id, run_info in run_info_dict.items():
        if run_info["input_run_id"] is None:
            main_flow_infos.append(run_info)
        else:
            input_run_id = run_info["input_run_id"]
            if input_run_id in run_info_dict:
                input_run_info = run_info_dict[input_run_id]
                updated_flow_artifacts = update_downstread_flow_artifacts(run_info["flow_artifacts"], run_info["display_name"])
                for line_number, updated_flow_artifact in updated_flow_artifacts.items():
                    if line_number in input_run_info["flow_artifacts"]:
                        input_run_info["flow_artifacts"][line_number].update(updated_flow_artifact)
            else:
                # Input Run is not included, treat this run as main flow
                main_flow_infos.append(run_info)
    
    merge_result = []
    for main_flow_info in main_flow_infos:
        merge_result += list(main_flow_info["flow_artifacts"].values())
    merge_result = sorted(merge_result, key=lambda x: f"{x['Line number']}{x['Run']}")
    return merge_result

# Convert the given jsonl file to csv file
def jsonl_to_csv(jsonl_path, csv_path):
    max_keys = []
    with open(jsonl_path, "r",encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            row = json.loads(line)
            if len(row.keys()) > len(max_keys):
                max_keys = list(row.keys())

    with open(jsonl_path, "r",encoding='utf-8') as jsonl_file:
        with open(csv_path, "w", newline="",encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(max_keys)
            for line in jsonl_file:
                row = json.loads(line)
                csv_row = [row.get(key) for key in max_keys]
                writer.writerow(csv_row)

## Download runs
Download jsonl of the runs, merge them and save to local jsonl_path and csv_path.

In [6]:
# The run_ids include all the runs selected from the UI.
run_ids = ["95baf68b-05a5-40c0-b7ec-e91d1ef30392"]

# Download the flow artifacts from the run_ids and merge them into a single file.
deduped_run_ids = list(set(run_ids))
run_infos = [get_required_info(run_id) for run_id in deduped_run_ids]
merged_result = merge_flow_artifacts(run_infos)

# Save the merged result as jsonl and csv.
jsonl_path = "./merged_result.jsonl"
csv_path = "./merged_result.csv"
with open(jsonl_path, "w") as fp:
    for record in merged_result:
        fp.write(json.dumps(record))
        fp.write("\n")
logger.info(f"Saved merged result as jsonl to {jsonl_path}")
jsonl_to_csv(jsonl_path, csv_path)
logger.info(f"Saved merged result as csv to {csv_path}")

2024-05-07 11:57:07,506 Processing Run: 95baf68b-05a5-40c0-b7ec-e91d1ef30392
2024-05-07 11:57:07,506 Getting Run Info for Run: 95baf68b-05a5-40c0-b7ec-e91d1ef30392
2024-05-07 11:57:23,473 Getting Flow Artifact Relative Path for Run 95baf68b-05a5-40c0-b7ec-e91d1ef30392
2024-05-07 11:57:23,473 Getting Output Asset Id for Run 95baf68b-05a5-40c0-b7ec-e91d1ef30392
2024-05-07 11:57:27,077 Getting Asset Path for Asset Id azureml://locations/eastus2/workspaces/c44e9724-0e65-4fa4-9b62-ebc0f390b962/data/azureml_95baf68b-05a5-40c0-b7ec-e91d1ef30392_output_data_debug_info/versions/1
2024-05-07 11:57:31,053 Downloading Flow Artifacts for Run 95baf68b-05a5-40c0-b7ec-e91d1ef30392


Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000000_000000024.jsonl, 1 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000275_000000299.jsonl, 2 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000100_000000124.jsonl, 3 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000075_000000099.jsonl, 4 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000125_000000149.jsonl, 5 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000250_000000274.jsonl, 6 files out of an estimated total of 659
Downloaded promptflow/Prompt

Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000675_000000699.jsonl, 30 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000775_000000799.jsonl, 31 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000825_000000849.jsonl, 32 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000725_000000749.jsonl, 33 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000975_000000999.jsonl, 34 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000000925_000000949.jsonl, 35 files out of an estimated total of 659
Downloaded promptflow/

Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001325_000001349.jsonl, 59 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001450_000001474.jsonl, 60 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001500_000001524.jsonl, 61 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001400_000001424.jsonl, 62 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001425_000001449.jsonl, 63 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000001550_000001574.jsonl, 64 files out of an estimated total of 659
Downloaded promptflow/

Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002075_000002099.jsonl, 88 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002375_000002399.jsonl, 89 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002175_000002199.jsonl, 90 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002200_000002224.jsonl, 91 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002250_000002274.jsonl, 92 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000002350_000002374.jsonl, 93 files out of an estimated total of 659
Downloaded promptflow/

...

Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015025_000015049.jsonl, 614 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015300_000015324.jsonl, 615 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015425_000015449.jsonl, 616 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015125_000015149.jsonl, 617 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015500_000015524.jsonl, 618 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015600_000015624.jsonl, 619 files out of an estimated total of 659
Downloaded promp

Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016025_000016049.jsonl, 643 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016250_000016274.jsonl, 644 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016200_000016224.jsonl, 645 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000015950_000015974.jsonl, 646 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016100_000016124.jsonl, 647 files out of an estimated total of 659
Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016150_000016174.jsonl, 648 files out of an estimated total of 659
Downloaded promp

2024-05-07 12:07:57,930 Loading Flow Artifacts of evaluation_ 1 from ./downloads/95baf68b-05a5-40c0-b7ec-e91d1ef30392


Downloaded promptflow/PromptFlowArtifacts/95baf68b-05a5-40c0-b7ec-e91d1ef30392/flow_artifacts/000016350_000016374.jsonl, 659 files out of an estimated total of 659


2024-05-07 12:08:33,948 Saved merged result as jsonl to ./merged_result.jsonl
2024-05-07 12:08:43,594 Saved merged result as csv to ./merged_result.csv


## Visualize data
Load local csv file and visualize it as pandas dataframe. Make sure you have installed pandas:

    pip install pandas

In [7]:
# Load the merged result as pandas dataframe and sort it by line number.
import pandas as pd
data = pd.read_csv('merged_result.csv', dtype={'Line number': 'int64'})
df = pd.DataFrame(data)
df.sort_values(by='Line number', ascending=True)

,Line number,Run,Status,inputs.question,inputs.answer,inputs.context,inputs.ground_truth,inputs.metrics,inputs.threshold,inputs.groundedness_service_flight,...,gpt_groundedness_reason,gpt_similarity,gpt_fluency,sexual,self_harm_score,hate_unfairness_reason,violence,hate_unfairness_score,self_harm_reason,f1_score
0,0,evaluation_ 1,Completed,Many people have car where they live. The thin...,Many people have car where they live. The thin...,Many people have car where they live. The thin...,NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7568,1,evaluation_ 1,Completed,I am a scientist at NASA that is discussing th...,I am a scientist at NASA that is discussing th...,I am a scientist at NASA that is discussing th...,NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8679,2,evaluation_ 1,Completed,People always wish they had the same technolog...,People always wish they had the same technolog...,People always wish they had the same technolog...,NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9790,3,evaluation_ 1,Completed,"We all heard about Venus, the planet without a...","We all heard about Venus, the planet without a...","We all heard about Venus, the planet without a...",NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10901,4,evaluation_ 1,Completed,"Dear, State Senator\n\nThis is a letter to arg...","Dear, State Senator\n\nThis is a letter to arg...","Dear, State Senator\n\nThis is a letter to arg...",NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7168,16452,evaluation_ 1,Completed,"Twenty five years ago, NASA's VIking spacecraf...","Twenty five years ago, NASA's VIking spacecraf...","Twenty five years ago, NASA's VIking spacecraf...",NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7169,16453,evaluation_ 1,Completed,I will like to argue against the development o...,I will like to argue against the development o...,I will like to argue against the development o...,NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7170,16454,evaluation_ 1,Completed,"The article ""The Challenge of Exploring Venus""...","The article ""The Challenge of Exploring Venus""...","The article ""The Challenge of Exploring Venus""...",NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7171,16455,evaluation_ 1,Completed,"From the crammed streets of New York City, to ...","From the crammed streets of New York City, to ...","From the crammed streets of New York City, to ...",NaN,"gpt_groundedness,gpt_relevance,gpt_coherence",4,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Potential Problems and Solutions
  
### 1. [Winerror] File name or path too long/The system cannot find the path specified 

**Solution: Enable NTFS long paths in Windows**

**For Windows Home Users: Filesystem Registry entry**
1. Press `Win + R` keys on your keyboard and type `regedit` then press `Enter`. The Registry Editor will be opened.
2. Go to `HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem`.
3. Create a new 32-bit DWORD value named `LongPathsEnabled` and set it to `1`.
4. Restart your Windows.

**For Windows Pro and Enterprise (Windows Server 2016 or above) Users: Using Group Policy**
1. Press `Win + R` keys on your keyboard and type `gpedit.msc` then press `Enter`. The Group Policy Editor will be opened.
2. Go to `Local Computer Policy -> Computer Configuration -> Administrative Templates -> System -> Filesystem`, then enable the `Enable Win32 long paths` option.
3. Restart your Windows.